In [1]:
!pip install transformers

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaModel
import torch.nn as nn # Add this import statement
import torch.optim as optim # Add this import statement
import torch.nn.functional as F # Add this import statement
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [3]:


# Load your dataset
interaction_data = pd.read_csv('/content/updated_interaction_data.csv')  # Load your data here

def map_rating_to_category(rating):
    if rating in [1, 2]:
        return 'negative'
    elif rating == 3:
        return 'neutral'
    elif rating in [4, 5]:
        return 'positive'
    else:
        raise ValueError("Rating out of range")

# Apply this function to your dataset
interaction_data['Category'] = interaction_data['Label'].apply(map_rating_to_category)
# Randomly sample 5,000 rows
sampled_data = interaction_data.sample(n=2000, random_state=42)

# Prepare dataset and dataloader


In [4]:
texts = sampled_data['Review'].tolist()  # Replace with your text column
labels = sampled_data['Category'].tolist()  # Replace with your labels # Replace with your labels

# Split dataset into 60% training and 40% testing
train_texts, test_texts, train_labels, test_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42
)

In [5]:

class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

        # Create a mapping from labels to integers
        self.label_map = {'negative': [0, 1],'neutral': [3],'positive': [4, 5]}
        #Define self.category_to_index
        self.category_to_index = {'negative': 0, 'neutral': 1, 'positive': 2}


    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            max_length=self.max_length,
            add_special_tokens=True,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        # Convert label to integer using the label map
        label =self.category_to_index[label]

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)  # Use long for classification
        }

# Initialize tokenizer and max length
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
max_length = 128  # Adjust based on your data

# Create datasets
train_dataset = CustomDataset(train_texts, train_labels, tokenizer, max_length)
test_dataset = CustomDataset(test_texts, test_labels, tokenizer, max_length)

# Create DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [9]:
class FineTuneRoberta(nn.Module):
    def __init__(self, model_name='roberta-base', n_classes=3): # Changed n_classes to 3
        super(FineTuneRoberta, self).__init__()
        self.model = RobertaModel.from_pretrained(model_name)
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.model.config.hidden_size, n_classes) # Output layer for 3 classes

    def forward(self, input_ids, attention_mask):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        output = self.drop(outputs.pooler_output)
        return self.out(output)
# Initialize model and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = FineTuneRoberta().to(device)
optimizer = optim.AdamW(model.parameters(), lr=2e-5)

def train_epoch(model, dataloader, optimizer, device):
    model.train()

    losses = []

    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = F.cross_entropy(outputs, labels)  # Use cross_entropy loss for classification
        losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return np.mean(losses)

# Training loop
for epoch in range(6):  # Adjust the number of epochs as needed
    train_loss = train_epoch(model, train_dataloader, optimizer, device)
    print(f'Epoch {epoch + 1}/{3}, Loss: {train_loss}')


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3, Loss: 0.374111523386091
Epoch 2/3, Loss: 0.215570800146088
Epoch 3/3, Loss: 0.14206411463674157
Epoch 4/3, Loss: 0.12194494540337474
Epoch 5/3, Loss: 0.08519389616558329
Epoch 6/3, Loss: 0.05617589584493544


In [8]:



def evaluate_model(model, dataloader, device):
    model.eval()  # Set the model to evaluation mode
    all_preds = []
    all_labels = []

    with torch.no_grad():  # No gradient calculation needed
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs, dim=1)  # Get the class with the highest probability

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Convert lists to numpy arrays for sklearn metrics
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)

    # Calculate accuracy
    accuracy = accuracy_score(all_labels, all_preds)

    # Calculate precision, recall, and F1 score
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')

    return accuracy, precision, recall, f1

# Assuming test_dataset is defined and you can replace it with your actual test dataset
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Evaluate the model
accuracy, precision, recall, f1 = evaluate_model(model, test_dataloader, device)

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')


Accuracy: 0.9150
Precision: 0.9106
Recall: 0.9150
F1 Score: 0.9044


In [10]:
# Save the entire model
torch.save(model, 'fine_tune_roberta_complete.pth')


In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score, precision_recall_fscore_support, confusion_matrix
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer, RobertaModel

In [12]:
# 1. Sentiment-Enhanced Collaborative Filtering with Temporal Dynamics
class TimeAwareSVD(nn.Module):
    def __init__(self, n_users, n_courses, n_factors=100, time_bins=30):
        super(TimeAwareSVD, self).__init__()
        self.user_factors = nn.Embedding(n_users, n_factors, max_norm=1)
        self.course_factors = nn.Embedding(n_courses, n_factors, max_norm=1)
        self.time_factors = nn.Embedding(time_bins, n_factors, max_norm=1)
        self.user_biases = nn.Embedding(n_users, 1)
        self.course_biases = nn.Embedding(n_courses, 1)
        self.time_biases = nn.Embedding(time_bins, 1)
        self.global_bias = nn.Parameter(torch.tensor([0.5]))  # Bias initialization

    def forward(self, user, course, time_bin):
        user_factor = self.user_factors(user)
        course_factor = self.course_factors(course)
        time_factor = self.time_factors(time_bin)

        user_bias = self.user_biases(user)
        course_bias = self.course_biases(course)
        time_bias = self.time_biases(time_bin)

        # Element-wise multiplication
        interaction_term = (user_factor * course_factor).sum(1) + (user_factor * time_factor).sum(1)

        pred = torch.sigmoid(self.global_bias + user_bias + course_bias + time_bias + interaction_term)
        return pred.squeeze()

# 2. Real-Time Contextual Awareness via Reinforcement Learning
class RLRecommendationAgent(nn.Module):
    def __init__(self, n_states, n_actions, n_hidden=128):
        super(RLRecommendationAgent, self).__init__()
        self.fc1 = nn.Linear(n_states, n_hidden)
        self.fc2 = nn.Linear(n_hidden, n_hidden)
        self.fc3 = nn.Linear(n_hidden, n_actions)

    def forward(self, state):
        x = torch.relu(self.fc1(state))
        x = torch.relu(self.fc2(x))
        action_values = self.fc3(x)
        return action_values

# Reinforcement Learning with Q-Learning
class QLearning:
    def __init__(self, agent, lr=0.001, gamma=0.99):
        self.agent = agent
        self.optimizer = optim.Adam(agent.parameters(), lr=lr)
        self.gamma = gamma

    def update(self, state, action, reward, next_state, done):
        q_values = self.agent(state)
        next_q_values = self.agent(next_state)

        q_value = q_values.gather(1, action.unsqueeze(1)).squeeze(1)
        next_q_value = next_q_values.max(1)[0]

        expected_q_value = reward + (1 - done) * self.gamma * next_q_value
        loss = F.mse_loss(q_value, expected_q_value.detach())

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

# 3. Emotion and Context-Driven Content-Based Filtering
class EmotionContextEmbeddingLayer(nn.Module):
    def __init__(self, model_name='roberta-base'):
        super(EmotionContextEmbeddingLayer, self).__init__()
        self.tokenizer = RobertaTokenizer.from_pretrained(model_name)
        self.model = RobertaModel.from_pretrained(model_name)

    def forward(self, text):
        inputs = self.tokenizer(text, return_tensors='pt', padding=True, truncation=True)
        outputs = self.model(**inputs)
        return outputs.last_hidden_state.mean(1)

# 4. Sequential Learning with Attention Mechanism Focused on Emotional Significance
class GRUWithAttention(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(GRUWithAttention, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, n_layers, batch_first=True)
        self.attn = nn.Linear(hidden_size, 1)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        gru_out, _ = self.gru(x)
        attn_weights = torch.softmax(self.attn(gru_out), dim=1)
        attn_applied = torch.sum(gru_out * attn_weights, dim=1)
        output = self.fc(attn_applied)
        return output

# 5. Fine-Tuned RoBERTa Model for Sentiment Analysis
class FineTuneRoberta(nn.Module):
    def __init__(self, model_name='roberta-base', n_classes=3):
        super(FineTuneRoberta, self).__init__()
        self.model = RobertaModel.from_pretrained(model_name)
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.model.config.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        output = self.drop(outputs.pooler_output)
        return self.out(output)

In [14]:
course_data = pd.read_csv('course_code_mapping.csv')
user_data = pd.read_csv('updated_studentInfo.csv')
interaction_data = pd.read_csv('updated_interaction_data.csv')

# Randomly sample 1000 rows from interaction_data
interaction_data_sample = interaction_data.sample(n=5000, random_state=42)

n_users = len(user_data['id_student'].unique())
n_courses = len(course_data['CourseCode'].unique())

In [15]:
time_aware_svd = TimeAwareSVD(n_users=n_users, n_courses=n_courses, n_factors=100)
rl_agent = RLRecommendationAgent(n_states=10, n_actions=n_courses)
emotion_context_layer = EmotionContextEmbeddingLayer()
gru_with_attention = GRUWithAttention(input_size=768, hidden_size=256, output_size=1)
fine_tuned_roberta = FineTuneRoberta(n_classes=3)

# Load the fine-tuned RoBERTa model state dict if available
model_path = '/content/fine_tune_roberta_complete.pth'
fine_tuned_roberta = torch.load(model_path)

# Prepare data for training and testing
train_data, test_data = train_test_split(interaction_data_sample, test_size=0.3, random_state=42)
train_labels = train_data['Label']
test_labels = test_data['Label']

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-15-cad32e0447b5>:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default valu

In [16]:
def get_model_predictions(model, data):
    model.eval()
    predictions = []
    for i, row in data.iterrows():
        user_idx = user_data[user_data['id_student'] == row['Id']].index[0]
        user_idx = min(user_idx, n_users - 1)
        course_idx = course_data[course_data['CourseCode'] == row['CourseCode']].index[0]

        user = torch.tensor([user_idx], dtype=torch.long)
        course = torch.tensor([course_idx], dtype=torch.long)
        time_bin = torch.tensor([row['timestamp'] % 30], dtype=torch.long)

        with torch.no_grad():
            if isinstance(model, TimeAwareSVD):
                pred = model(user, course, time_bin)
                predictions.append(pred.item())
            elif isinstance(model, RLRecommendationAgent):
                # Placeholder for RL predictions (need actual state features)
                pred = model(torch.rand(10))
                predictions.append(pred.mean().item())
            elif isinstance(model, EmotionContextEmbeddingLayer):
                text = row['Review']
                inputs = emotion_context_layer.tokenizer(text, return_tensors='pt', padding=True, truncation=True)
                outputs = emotion_context_layer.model(**inputs)
                pred = outputs.last_hidden_state.mean(1).numpy()
                predictions.append(pred.mean())
            elif isinstance(model, GRUWithAttention):
                # Placeholder for GRU predictions (need actual sequence data)
                pred = model(torch.rand(1, 10, 768))
                predictions.append(pred.item())
            elif isinstance(model, FineTuneRoberta):
                text = row['Review']
                inputs = emotion_context_layer.tokenizer(text, return_tensors='pt', padding=True, truncation=True)
                with torch.no_grad():
                    logits = model(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'])
                pred = torch.argmax(logits, dim=1).item()
                predictions.append(pred)
    return np.array(predictions)

In [5]:
!pip install torch

In [20]:
import torch
import torch.optim as optim
def train_base_models():
    # Train TimeAwareSVD
    optimizer = optim.Adam(time_aware_svd.parameters(), lr=0.001)
    for epoch in range(10):
        epoch_loss = 0
        for i, row in train_data.iterrows():
            user_idx = user_data[user_data['id_student'] == row['Id']].index[0]
            user_idx = min(user_idx, n_users - 1)
            course_idx = course_data[course_data['CourseCode'] == row['CourseCode']].index[0]
            user = torch.tensor([user_idx], dtype=torch.long)
            course = torch.tensor([course_idx], dtype=torch.long)
            time_bin = torch.tensor([row['timestamp'] % 30], dtype=torch.long)

            svd_pred = time_aware_svd(user, course, time_bin)
            target = torch.tensor([row['Label']], dtype=torch.float32)
            loss = F.mse_loss(svd_pred, target)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {epoch_loss/len(train_data)}")

In [ ]:
train_base_models()

# Generate base model predictions
train_preds_svd = get_model_predictions(time_aware_svd, train_data)
test_preds_svd = get_model_predictions(time_aware_svd, test_data)

train_preds_rl = get_model_predictions(rl_agent, train_data)
test_preds_rl = get_model_predictions(rl_agent, test_data)

train_preds_emotion = get_model_predictions(emotion_context_layer, train_data)
test_preds_emotion = get_model_predictions(emotion_context_layer, test_data)

train_preds_gru = get_model_predictions(gru_with_attention, train_data)
test_preds_gru = get_model_predictions(gru_with_attention, test_data)

train_preds_roberta = get_model_predictions(fine_tuned_roberta, train_data)
test_preds_roberta = get_model_predictions(fine_tuned_roberta, test_data)

# Stack predictions
train_base_preds = np.column_stack((train_preds_svd, train_preds_rl, train_preds_emotion, train_preds_gru, train_preds_roberta))
test_base_preds = np.column_stack((test_preds_svd, test_preds_rl, test_preds_emotion, test_preds_gru, test_preds_roberta))

# Train meta-model
meta_model = LogisticRegression()
meta_model.fit(train_base_preds, train_labels)

# Evaluate
final_predictions = meta_model.predict(test_base_preds)

# Metrics
rmse = np.sqrt(mean_squared_error(test_labels, final_predictions))
mae = mean_absolute_error(test_labels, final_predictions)
accuracy = accuracy_score(test_labels, final_predictions)
precision, recall, f1, _ = precision_recall_fscore_support(test_labels, final_predictions, average='weighted')  # Use 'weighted' for multiclass
conf_matrix = confusion_matrix(test_labels, final_predictions)

print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")
print("Confusion Matrix:")
print(conf_matrix)

<ipython-input-20-c553dfad2b39>:18: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(svd_pred, target)


Epoch 1, Loss: 14.794568545241127
Epoch 2, Loss: 13.733864458110142
Epoch 3, Loss: 13.59087210785429
Epoch 4, Loss: 13.560203216003849
Epoch 5, Loss: 13.551587282011953
Epoch 6, Loss: 13.548864182698704
Epoch 7, Loss: 13.54794493915635
Epoch 8, Loss: 13.547620375485314
Epoch 9, Loss: 13.547501668044289
Epoch 10, Loss: 13.547456975199735
